### Advent of Code: Day 3

### Part One

#### My process: 
- Convert each `#123 @ 3,2: 5x4` line into a tuple marking the upper-left and lower-right corners.
- Create a dictionary with each key representing a pixel.
- Instantiate an overlap counter.
- For each key in the dictionary, start counting up the number of instances the pixel is in between the upper-left and lower-right corners.
    - This would get us to a workable solution; it can be sped up by only counting if a pixel is covered by at least two areas, then move onto the next pixel rather than counting every overlap.
- Return the count of the pixels for which this is true.

#### Solution:

In [ ]:
def transform(code):
    code = code.replace('@ ','---').replace(': ','---').split('---')
    upper_left_x = int(code[1].split(',')[0])
    upper_left_y = int(code[1].split(',')[1])
    bottom_right_x = upper_left_x - 1 + int(code[2].split('x')[0])
    bottom_right_y = upper_left_y - 1 + int(code[2].split('x')[1])
    return (upper_left_x, upper_left_y, bottom_right_x, bottom_right_y)

In [1]:
# Start timer.
import time
t0 = time.time()

# Import file.
file = open('./day_3_input.txt', 'r')

# Instantiate overlap counter.
overlap_counter = 0

# Instantiate pixel_dict dictionary.
pixel_dict = {(i,j): 0 for i in range(1000) for j in range(1000)}

# Instantiate code counter to track how quickly I'm going.
code_count = 0

# Iterate through each item in our input.
for code in file:
    
    # Transform our input into coordinates.
    coords = transform(code)

    # Iterate through each item in pixel_dict.
    for pixel in pixel_dict.keys():
    
        # Check if the pixel is in the columns to be cut.
        if pixel[0] >= coords[0]:
            if pixel[0] <= coords[2]:
                if pixel[1] >= coords[1]:
                    if pixel[1] <= coords[3]:
                        
                        # If the pixel is in the columns and
                        # rows to be cut, add one to count.
                        pixel_dict[pixel] += 1

    code_count += 1

    # Check to make sure we're making progress. 
    if code_count % 25 == 0:
        print(f'We finished claim {code_count} of 1,233 after {round(time.time()-t0,1)} seconds.')
        
for pixel in pixel_dict.keys():
    if pixel_dict[pixel] >= 2:
        overlap_counter += 1

maximum = 0
for v in pixel_dict.values():
    if v > maximum:
        maximum = v

print(maximum)
print(overlap_counter)

### Part Two

#### My process: 
- I will loop through each code snippet and see if a corner of my first code snippet is inside any other code snippet. (Update: this turned out to be wrong.)

In [ ]:
# Start timer.
import time
t0 = time.time()

# Import file.
file = open('./day_3_input.txt', 'r')

file_list = [code for code in file]

lst = []

# Iterate through file.
for i in range(len(file_list)):
    
    # Retrieve coordinates of i.
    i_coords = transform(file_list[i])
    
# (i_coords[0], i_coords[1]) is the upper-left corner.
# (i_coords[0], i_coords[3]) is the lower-left corner.
# (i_coords[2], i_coords[3]) is the lower-right corner.
# (i_coords[2], i_coords[1]) is the upper-right corner.
    
    # Iterate through remainder of file.
    for j in file_list[i+1:]:
        
        # Retrieve coordinates of j.
        j_coords = transform(j)
        
        # Check if upper-left corner of i is inside j.
        if i_coords[0] >= j_coords[0]:
            if i_coords[0] <= j_coords[2]:
                if i_coords[1] >= j_coords[1]:
                    if i_coords[1] <= j_coords[3]:
                        break
        
        # Check if lower-left corner of i is inside j.
        if i_coords[0] >= j_coords[0]:
            if i_coords[0] <= j_coords[2]:
                if i_coords[3] >= j_coords[1]:
                    if i_coords[3] <= j_coords[3]:
                        break
        
        # Check if upper-right corner of i is inside j.
        if i_coords[2] >= j_coords[0]:
            if i_coords[2] <= j_coords[2]:
                if i_coords[1] >= j_coords[1]:
                    if i_coords[1] <= j_coords[3]:
                        break
                        
        # Check if lower-right corner of i is inside j.
        if i_coords[2] >= j_coords[0]:
            if i_coords[2] <= j_coords[2]:
                if i_coords[3] >= j_coords[1]:
                    if i_coords[3] <= j_coords[3]:
                        break
                        
        if j == file_list[-1]:
            lst.append(i)
    
    if i % 100 == 0:
        print(f'We are through iteration {i} of 1,233 after {round(time.time()-t0,3)} seconds.')
        
print(f'Our list of candidates is {lst}.')

for k in lst:
    coord_1, coord_2, coord_3, coord_4 = transform(file_list[k])
    count = 0
    for x in range(coord_1, coord_3 + 1):
        for y in range(coord_2, coord_4 + 1):
            if pixel_dict[x,y] == 1:
                count += 1
    if count == (coord_3 - coord_1 + 1) * (coord_4 - coord_2 + 1):
        print(k)

#### How might I improve this?
I wonder if there's a way to just check if a string contains two or three values, in case the strings were really long or there were millions of rows.

#### Here's a sandbox of some things I explored to make sure they worked the way I expected:

I want to split my string on the `@` and the `:`.

In [5]:
'#123 @ 3,2: 5x4'.split('@ ')

['#123 ', '3,2: 5x4']

In [6]:
'#123 @ 3,2: 5x4'.split(['@ ', ': '])

TypeError: must be str or None, not list

[This site](https://stackoverflow.com/questions/1059559/split-strings-with-multiple-delimiters) got me the answer: replace the items I want to find to some other string, then split on *that new string*.

In [8]:
'#123 @ 3,2: 5x4'.replace('@ ','---').replace(': ','---').split('---')

['#123 ', '3,2', '5x4']

In [18]:
'#123 @ 3,2: 5x4'.replace('@ ','---').replace(': ','---').split('---')[1].split(',')

['3', '2']

In [19]:
int('#123 @ 3,2: 5x4'.replace('@ ','---').replace(': ','---').split('---')[1].split(',')[0])

3

In [14]:
'#123 @ 3,2: 5x4'.replace('@ ','---').replace(': ','---').split('---')[2].split('x')

['5', '4']

Let's write a function taking in one code snippet and returning the upper-left and bottom-right coordinates of the cut fabric. Note that I only need those two corners to define the whole region!

In [82]:
def transform(code):
    code = code.replace('@ ','---').replace(': ','---').split('---')
    upper_left_x = int(code[1].split(',')[0])
    upper_left_y = int(code[1].split(',')[1])
    bottom_right_x = upper_left_x - 1 + int(code[2].split('x')[0])
    bottom_right_y = upper_left_y - 1 + int(code[2].split('x')[1])
    return (upper_left_x, upper_left_y, bottom_right_x, bottom_right_y)

In [90]:
transform('#2 @ 3,1: 4x4')

(3, 1, 6, 4)

In order to make my dictionary, I have to figure out how many rows and columns there are in the fabric!

In [72]:
file = open('./day_3_input.txt', 'r')

max_row = 0
max_column = 0

for i in file:
    coords = transform(i)
    if coords[2] > max_row:
        max_row = coords[2]
    if coords[3] > max_column:
        max_column = coords[3]
        
print(max_row, max_column)

999 999


The prompt said "at least 1000" inches on each side... and now we know we don't have to go beyond that!

In [73]:
# I started to do nested for loops and instead
# decided to do a dictionary comprehension!
pixel_dict = {(i,j): 0 for i in range(1000) for j in range(1000)}

In [74]:
pixel_dict

{(0, 0): 0,
 (0, 1): 0,
 (0, 2): 0,
 (0, 3): 0,
 (0, 4): 0,
 (0, 5): 0,
 (0, 6): 0,
 (0, 7): 0,
 (0, 8): 0,
 (0, 9): 0,
 (0, 10): 0,
 (0, 11): 0,
 (0, 12): 0,
 (0, 13): 0,
 (0, 14): 0,
 (0, 15): 0,
 (0, 16): 0,
 (0, 17): 0,
 (0, 18): 0,
 (0, 19): 0,
 (0, 20): 0,
 (0, 21): 0,
 (0, 22): 0,
 (0, 23): 0,
 (0, 24): 0,
 (0, 25): 0,
 (0, 26): 0,
 (0, 27): 0,
 (0, 28): 0,
 (0, 29): 0,
 (0, 30): 0,
 (0, 31): 0,
 (0, 32): 0,
 (0, 33): 0,
 (0, 34): 0,
 (0, 35): 0,
 (0, 36): 0,
 (0, 37): 0,
 (0, 38): 0,
 (0, 39): 0,
 (0, 40): 0,
 (0, 41): 0,
 (0, 42): 0,
 (0, 43): 0,
 (0, 44): 0,
 (0, 45): 0,
 (0, 46): 0,
 (0, 47): 0,
 (0, 48): 0,
 (0, 49): 0,
 (0, 50): 0,
 (0, 51): 0,
 (0, 52): 0,
 (0, 53): 0,
 (0, 54): 0,
 (0, 55): 0,
 (0, 56): 0,
 (0, 57): 0,
 (0, 58): 0,
 (0, 59): 0,
 (0, 60): 0,
 (0, 61): 0,
 (0, 62): 0,
 (0, 63): 0,
 (0, 64): 0,
 (0, 65): 0,
 (0, 66): 0,
 (0, 67): 0,
 (0, 68): 0,
 (0, 69): 0,
 (0, 70): 0,
 (0, 71): 0,
 (0, 72): 0,
 (0, 73): 0,
 (0, 74): 0,
 (0, 75): 0,
 (0, 76): 0,
 (0, 77):

In [36]:
len(pixel_dict)

1000000

In [44]:
for pixel, count in pixel_dict.items():
    print(pixel)

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(0, 16)
(0, 17)
(0, 18)
(0, 19)
(0, 20)
(0, 21)
(0, 22)
(0, 23)
(0, 24)
(0, 25)
(0, 26)
(0, 27)
(0, 28)
(0, 29)
(0, 30)
(0, 31)
(0, 32)
(0, 33)
(0, 34)
(0, 35)
(0, 36)
(0, 37)
(0, 38)
(0, 39)
(0, 40)
(0, 41)
(0, 42)
(0, 43)
(0, 44)
(0, 45)
(0, 46)
(0, 47)
(0, 48)
(0, 49)
(0, 50)
(0, 51)
(0, 52)
(0, 53)
(0, 54)
(0, 55)
(0, 56)
(0, 57)
(0, 58)
(0, 59)
(0, 60)
(0, 61)
(0, 62)
(0, 63)
(0, 64)
(0, 65)
(0, 66)
(0, 67)
(0, 68)
(0, 69)
(0, 70)
(0, 71)
(0, 72)
(0, 73)
(0, 74)
(0, 75)
(0, 76)
(0, 77)
(0, 78)
(0, 79)
(0, 80)
(0, 81)
(0, 82)
(0, 83)
(0, 84)
(0, 85)
(0, 86)
(0, 87)
(0, 88)
(0, 89)
(0, 90)
(0, 91)
(0, 92)
(0, 93)
(0, 94)
(0, 95)
(0, 96)
(0, 97)
(0, 98)
(0, 99)
(0, 100)
(0, 101)
(0, 102)
(0, 103)
(0, 104)
(0, 105)
(0, 106)
(0, 107)
(0, 108)
(0, 109)
(0, 110)
(0, 111)
(0, 112)
(0, 113)
(0, 114)
(0, 115)
(0, 116)
(0, 117)
(0, 118)
(0, 119)
(0, 120)
(0, 121)
(0, 122)
(0,

(1, 959)
(1, 960)
(1, 961)
(1, 962)
(1, 963)
(1, 964)
(1, 965)
(1, 966)
(1, 967)
(1, 968)
(1, 969)
(1, 970)
(1, 971)
(1, 972)
(1, 973)
(1, 974)
(1, 975)
(1, 976)
(1, 977)
(1, 978)
(1, 979)
(1, 980)
(1, 981)
(1, 982)
(1, 983)
(1, 984)
(1, 985)
(1, 986)
(1, 987)
(1, 988)
(1, 989)
(1, 990)
(1, 991)
(1, 992)
(1, 993)
(1, 994)
(1, 995)
(1, 996)
(1, 997)
(1, 998)
(1, 999)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(2, 7)
(2, 8)
(2, 9)
(2, 10)
(2, 11)
(2, 12)
(2, 13)
(2, 14)
(2, 15)
(2, 16)
(2, 17)
(2, 18)
(2, 19)
(2, 20)
(2, 21)
(2, 22)
(2, 23)
(2, 24)
(2, 25)
(2, 26)
(2, 27)
(2, 28)
(2, 29)
(2, 30)
(2, 31)
(2, 32)
(2, 33)
(2, 34)
(2, 35)
(2, 36)
(2, 37)
(2, 38)
(2, 39)
(2, 40)
(2, 41)
(2, 42)
(2, 43)
(2, 44)
(2, 45)
(2, 46)
(2, 47)
(2, 48)
(2, 49)
(2, 50)
(2, 51)
(2, 52)
(2, 53)
(2, 54)
(2, 55)
(2, 56)
(2, 57)
(2, 58)
(2, 59)
(2, 60)
(2, 61)
(2, 62)
(2, 63)
(2, 64)
(2, 65)
(2, 66)
(2, 67)
(2, 68)
(2, 69)
(2, 70)
(2, 71)
(2, 72)
(2, 73)
(2, 74)
(2, 75)
(2, 76)
(2, 77)
(2, 78)
(2, 79)
(

(3, 458)
(3, 459)
(3, 460)
(3, 461)
(3, 462)
(3, 463)
(3, 464)
(3, 465)
(3, 466)
(3, 467)
(3, 468)
(3, 469)
(3, 470)
(3, 471)
(3, 472)
(3, 473)
(3, 474)
(3, 475)
(3, 476)
(3, 477)
(3, 478)
(3, 479)
(3, 480)
(3, 481)
(3, 482)
(3, 483)
(3, 484)
(3, 485)
(3, 486)
(3, 487)
(3, 488)
(3, 489)
(3, 490)
(3, 491)
(3, 492)
(3, 493)
(3, 494)
(3, 495)
(3, 496)
(3, 497)
(3, 498)
(3, 499)
(3, 500)
(3, 501)
(3, 502)
(3, 503)
(3, 504)
(3, 505)
(3, 506)
(3, 507)
(3, 508)
(3, 509)
(3, 510)
(3, 511)
(3, 512)
(3, 513)
(3, 514)
(3, 515)
(3, 516)
(3, 517)
(3, 518)
(3, 519)
(3, 520)
(3, 521)
(3, 522)
(3, 523)
(3, 524)
(3, 525)
(3, 526)
(3, 527)
(3, 528)
(3, 529)
(3, 530)
(3, 531)
(3, 532)
(3, 533)
(3, 534)
(3, 535)
(3, 536)
(3, 537)
(3, 538)
(3, 539)
(3, 540)
(3, 541)
(3, 542)
(3, 543)
(3, 544)
(3, 545)
(3, 546)
(3, 547)
(3, 548)
(3, 549)
(3, 550)
(3, 551)
(3, 552)
(3, 553)
(3, 554)
(3, 555)
(3, 556)
(3, 557)
(3, 558)
(3, 559)
(3, 560)
(3, 561)
(3, 562)
(3, 563)
(3, 564)
(3, 565)
(3, 566)
(3, 567)
(3, 568)
(

(5, 458)
(5, 459)
(5, 460)
(5, 461)
(5, 462)
(5, 463)
(5, 464)
(5, 465)
(5, 466)
(5, 467)
(5, 468)
(5, 469)
(5, 470)
(5, 471)
(5, 472)
(5, 473)
(5, 474)
(5, 475)
(5, 476)
(5, 477)
(5, 478)
(5, 479)
(5, 480)
(5, 481)
(5, 482)
(5, 483)
(5, 484)
(5, 485)
(5, 486)
(5, 487)
(5, 488)
(5, 489)
(5, 490)
(5, 491)
(5, 492)
(5, 493)
(5, 494)
(5, 495)
(5, 496)
(5, 497)
(5, 498)
(5, 499)
(5, 500)
(5, 501)
(5, 502)
(5, 503)
(5, 504)
(5, 505)
(5, 506)
(5, 507)
(5, 508)
(5, 509)
(5, 510)
(5, 511)
(5, 512)
(5, 513)
(5, 514)
(5, 515)
(5, 516)
(5, 517)
(5, 518)
(5, 519)
(5, 520)
(5, 521)
(5, 522)
(5, 523)
(5, 524)
(5, 525)
(5, 526)
(5, 527)
(5, 528)
(5, 529)
(5, 530)
(5, 531)
(5, 532)
(5, 533)
(5, 534)
(5, 535)
(5, 536)
(5, 537)
(5, 538)
(5, 539)
(5, 540)
(5, 541)
(5, 542)
(5, 543)
(5, 544)
(5, 545)
(5, 546)
(5, 547)
(5, 548)
(5, 549)
(5, 550)
(5, 551)
(5, 552)
(5, 553)
(5, 554)
(5, 555)
(5, 556)
(5, 557)
(5, 558)
(5, 559)
(5, 560)
(5, 561)
(5, 562)
(5, 563)
(5, 564)
(5, 565)
(5, 566)
(5, 567)
(5, 568)
(

(7, 457)
(7, 458)
(7, 459)
(7, 460)
(7, 461)
(7, 462)
(7, 463)
(7, 464)
(7, 465)
(7, 466)
(7, 467)
(7, 468)
(7, 469)
(7, 470)
(7, 471)
(7, 472)
(7, 473)
(7, 474)
(7, 475)
(7, 476)
(7, 477)
(7, 478)
(7, 479)
(7, 480)
(7, 481)
(7, 482)
(7, 483)
(7, 484)
(7, 485)
(7, 486)
(7, 487)
(7, 488)
(7, 489)
(7, 490)
(7, 491)
(7, 492)
(7, 493)
(7, 494)
(7, 495)
(7, 496)
(7, 497)
(7, 498)
(7, 499)
(7, 500)
(7, 501)
(7, 502)
(7, 503)
(7, 504)
(7, 505)
(7, 506)
(7, 507)
(7, 508)
(7, 509)
(7, 510)
(7, 511)
(7, 512)
(7, 513)
(7, 514)
(7, 515)
(7, 516)
(7, 517)
(7, 518)
(7, 519)
(7, 520)
(7, 521)
(7, 522)
(7, 523)
(7, 524)
(7, 525)
(7, 526)
(7, 527)
(7, 528)
(7, 529)
(7, 530)
(7, 531)
(7, 532)
(7, 533)
(7, 534)
(7, 535)
(7, 536)
(7, 537)
(7, 538)
(7, 539)
(7, 540)
(7, 541)
(7, 542)
(7, 543)
(7, 544)
(7, 545)
(7, 546)
(7, 547)
(7, 548)
(7, 549)
(7, 550)
(7, 551)
(7, 552)
(7, 553)
(7, 554)
(7, 555)
(7, 556)
(7, 557)
(7, 558)
(7, 559)
(7, 560)
(7, 561)
(7, 562)
(7, 563)
(7, 564)
(7, 565)
(7, 566)
(7, 567)
(

(9, 457)
(9, 458)
(9, 459)
(9, 460)
(9, 461)
(9, 462)
(9, 463)
(9, 464)
(9, 465)
(9, 466)
(9, 467)
(9, 468)
(9, 469)
(9, 470)
(9, 471)
(9, 472)
(9, 473)
(9, 474)
(9, 475)
(9, 476)
(9, 477)
(9, 478)
(9, 479)
(9, 480)
(9, 481)
(9, 482)
(9, 483)
(9, 484)
(9, 485)
(9, 486)
(9, 487)
(9, 488)
(9, 489)
(9, 490)
(9, 491)
(9, 492)
(9, 493)
(9, 494)
(9, 495)
(9, 496)
(9, 497)
(9, 498)
(9, 499)
(9, 500)
(9, 501)
(9, 502)
(9, 503)
(9, 504)
(9, 505)
(9, 506)
(9, 507)
(9, 508)
(9, 509)
(9, 510)
(9, 511)
(9, 512)
(9, 513)
(9, 514)
(9, 515)
(9, 516)
(9, 517)
(9, 518)
(9, 519)
(9, 520)
(9, 521)
(9, 522)
(9, 523)
(9, 524)
(9, 525)
(9, 526)
(9, 527)
(9, 528)
(9, 529)
(9, 530)
(9, 531)
(9, 532)
(9, 533)
(9, 534)
(9, 535)
(9, 536)
(9, 537)
(9, 538)
(9, 539)
(9, 540)
(9, 541)
(9, 542)
(9, 543)
(9, 544)
(9, 545)
(9, 546)
(9, 547)
(9, 548)
(9, 549)
(9, 550)
(9, 551)
(9, 552)
(9, 553)
(9, 554)
(9, 555)
(9, 556)
(9, 557)
(9, 558)
(9, 559)
(9, 560)
(9, 561)
(9, 562)
(9, 563)
(9, 564)
(9, 565)
(9, 566)
(9, 567)
(

(10, 956)
(10, 957)
(10, 958)
(10, 959)
(10, 960)
(10, 961)
(10, 962)
(10, 963)
(10, 964)
(10, 965)
(10, 966)
(10, 967)
(10, 968)
(10, 969)
(10, 970)
(10, 971)
(10, 972)
(10, 973)
(10, 974)
(10, 975)
(10, 976)
(10, 977)
(10, 978)
(10, 979)
(10, 980)
(10, 981)
(10, 982)
(10, 983)
(10, 984)
(10, 985)
(10, 986)
(10, 987)
(10, 988)
(10, 989)
(10, 990)
(10, 991)
(10, 992)
(10, 993)
(10, 994)
(10, 995)
(10, 996)
(10, 997)
(10, 998)
(10, 999)
(11, 0)
(11, 1)
(11, 2)
(11, 3)
(11, 4)
(11, 5)
(11, 6)
(11, 7)
(11, 8)
(11, 9)
(11, 10)
(11, 11)
(11, 12)
(11, 13)
(11, 14)
(11, 15)
(11, 16)
(11, 17)
(11, 18)
(11, 19)
(11, 20)
(11, 21)
(11, 22)
(11, 23)
(11, 24)
(11, 25)
(11, 26)
(11, 27)
(11, 28)
(11, 29)
(11, 30)
(11, 31)
(11, 32)
(11, 33)
(11, 34)
(11, 35)
(11, 36)
(11, 37)
(11, 38)
(11, 39)
(11, 40)
(11, 41)
(11, 42)
(11, 43)
(11, 44)
(11, 45)
(11, 46)
(11, 47)
(11, 48)
(11, 49)
(11, 50)
(11, 51)
(11, 52)
(11, 53)
(11, 54)
(11, 55)
(11, 56)
(11, 57)
(11, 58)
(11, 59)
(11, 60)
(11, 61)
(11, 62)
(11

(12, 956)
(12, 957)
(12, 958)
(12, 959)
(12, 960)
(12, 961)
(12, 962)
(12, 963)
(12, 964)
(12, 965)
(12, 966)
(12, 967)
(12, 968)
(12, 969)
(12, 970)
(12, 971)
(12, 972)
(12, 973)
(12, 974)
(12, 975)
(12, 976)
(12, 977)
(12, 978)
(12, 979)
(12, 980)
(12, 981)
(12, 982)
(12, 983)
(12, 984)
(12, 985)
(12, 986)
(12, 987)
(12, 988)
(12, 989)
(12, 990)
(12, 991)
(12, 992)
(12, 993)
(12, 994)
(12, 995)
(12, 996)
(12, 997)
(12, 998)
(12, 999)
(13, 0)
(13, 1)
(13, 2)
(13, 3)
(13, 4)
(13, 5)
(13, 6)
(13, 7)
(13, 8)
(13, 9)
(13, 10)
(13, 11)
(13, 12)
(13, 13)
(13, 14)
(13, 15)
(13, 16)
(13, 17)
(13, 18)
(13, 19)
(13, 20)
(13, 21)
(13, 22)
(13, 23)
(13, 24)
(13, 25)
(13, 26)
(13, 27)
(13, 28)
(13, 29)
(13, 30)
(13, 31)
(13, 32)
(13, 33)
(13, 34)
(13, 35)
(13, 36)
(13, 37)
(13, 38)
(13, 39)
(13, 40)
(13, 41)
(13, 42)
(13, 43)
(13, 44)
(13, 45)
(13, 46)
(13, 47)
(13, 48)
(13, 49)
(13, 50)
(13, 51)
(13, 52)
(13, 53)
(13, 54)
(13, 55)
(13, 56)
(13, 57)
(13, 58)
(13, 59)
(13, 60)
(13, 61)
(13, 62)
(13

(14, 955)
(14, 956)
(14, 957)
(14, 958)
(14, 959)
(14, 960)
(14, 961)
(14, 962)
(14, 963)
(14, 964)
(14, 965)
(14, 966)
(14, 967)
(14, 968)
(14, 969)
(14, 970)
(14, 971)
(14, 972)
(14, 973)
(14, 974)
(14, 975)
(14, 976)
(14, 977)
(14, 978)
(14, 979)
(14, 980)
(14, 981)
(14, 982)
(14, 983)
(14, 984)
(14, 985)
(14, 986)
(14, 987)
(14, 988)
(14, 989)
(14, 990)
(14, 991)
(14, 992)
(14, 993)
(14, 994)
(14, 995)
(14, 996)
(14, 997)
(14, 998)
(14, 999)
(15, 0)
(15, 1)
(15, 2)
(15, 3)
(15, 4)
(15, 5)
(15, 6)
(15, 7)
(15, 8)
(15, 9)
(15, 10)
(15, 11)
(15, 12)
(15, 13)
(15, 14)
(15, 15)
(15, 16)
(15, 17)
(15, 18)
(15, 19)
(15, 20)
(15, 21)
(15, 22)
(15, 23)
(15, 24)
(15, 25)
(15, 26)
(15, 27)
(15, 28)
(15, 29)
(15, 30)
(15, 31)
(15, 32)
(15, 33)
(15, 34)
(15, 35)
(15, 36)
(15, 37)
(15, 38)
(15, 39)
(15, 40)
(15, 41)
(15, 42)
(15, 43)
(15, 44)
(15, 45)
(15, 46)
(15, 47)
(15, 48)
(15, 49)
(15, 50)
(15, 51)
(15, 52)
(15, 53)
(15, 54)
(15, 55)
(15, 56)
(15, 57)
(15, 58)
(15, 59)
(15, 60)
(15, 61)
(1

KeyboardInterrupt: 

Initially, I was going to just check if `pixel_dict[pixel][0] >= coords[0] and pixel_dict[pixel][0] <= coords[2] and pixel_dict[pixel][1] >= coords[1] and pixel_dict[pixel][1] <= coords[3]`. However, this means that at each step I need to check four conditions and see if they all evaluate to `True`. By nesting these, if one condition isn't met, we can skip all subsequent checks and just move onto the next item in our file.

In [68]:
for pixel in pixel_dict.keys():
    print(pixel)

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(0, 16)
(0, 17)
(0, 18)
(0, 19)
(0, 20)
(0, 21)
(0, 22)
(0, 23)
(0, 24)
(0, 25)
(0, 26)
(0, 27)
(0, 28)
(0, 29)
(0, 30)
(0, 31)
(0, 32)
(0, 33)
(0, 34)
(0, 35)
(0, 36)
(0, 37)
(0, 38)
(0, 39)
(0, 40)
(0, 41)
(0, 42)
(0, 43)
(0, 44)
(0, 45)
(0, 46)
(0, 47)
(0, 48)
(0, 49)
(0, 50)
(0, 51)
(0, 52)
(0, 53)
(0, 54)
(0, 55)
(0, 56)
(0, 57)
(0, 58)
(0, 59)
(0, 60)
(0, 61)
(0, 62)
(0, 63)
(0, 64)
(0, 65)
(0, 66)
(0, 67)
(0, 68)
(0, 69)
(0, 70)
(0, 71)
(0, 72)
(0, 73)
(0, 74)
(0, 75)
(0, 76)
(0, 77)
(0, 78)
(0, 79)
(0, 80)
(0, 81)
(0, 82)
(0, 83)
(0, 84)
(0, 85)
(0, 86)
(0, 87)
(0, 88)
(0, 89)
(0, 90)
(0, 91)
(0, 92)
(0, 93)
(0, 94)
(0, 95)
(0, 96)
(0, 97)
(0, 98)
(0, 99)
(0, 100)
(0, 101)
(0, 102)
(0, 103)
(0, 104)
(0, 105)
(0, 106)
(0, 107)
(0, 108)
(0, 109)
(0, 110)
(0, 111)
(0, 112)
(0, 113)
(0, 114)
(0, 115)
(0, 116)
(0, 117)
(0, 118)
(0, 119)
(0, 120)
(0, 121)
(0, 122)
(0,

(2, 144)
(2, 145)
(2, 146)
(2, 147)
(2, 148)
(2, 149)
(2, 150)
(2, 151)
(2, 152)
(2, 153)
(2, 154)
(2, 155)
(2, 156)
(2, 157)
(2, 158)
(2, 159)
(2, 160)
(2, 161)
(2, 162)
(2, 163)
(2, 164)
(2, 165)
(2, 166)
(2, 167)
(2, 168)
(2, 169)
(2, 170)
(2, 171)
(2, 172)
(2, 173)
(2, 174)
(2, 175)
(2, 176)
(2, 177)
(2, 178)
(2, 179)
(2, 180)
(2, 181)
(2, 182)
(2, 183)
(2, 184)
(2, 185)
(2, 186)
(2, 187)
(2, 188)
(2, 189)
(2, 190)
(2, 191)
(2, 192)
(2, 193)
(2, 194)
(2, 195)
(2, 196)
(2, 197)
(2, 198)
(2, 199)
(2, 200)
(2, 201)
(2, 202)
(2, 203)
(2, 204)
(2, 205)
(2, 206)
(2, 207)
(2, 208)
(2, 209)
(2, 210)
(2, 211)
(2, 212)
(2, 213)
(2, 214)
(2, 215)
(2, 216)
(2, 217)
(2, 218)
(2, 219)
(2, 220)
(2, 221)
(2, 222)
(2, 223)
(2, 224)
(2, 225)
(2, 226)
(2, 227)
(2, 228)
(2, 229)
(2, 230)
(2, 231)
(2, 232)
(2, 233)
(2, 234)
(2, 235)
(2, 236)
(2, 237)
(2, 238)
(2, 239)
(2, 240)
(2, 241)
(2, 242)
(2, 243)
(2, 244)
(2, 245)
(2, 246)
(2, 247)
(2, 248)
(2, 249)
(2, 250)
(2, 251)
(2, 252)
(2, 253)
(2, 254)
(

(3, 643)
(3, 644)
(3, 645)
(3, 646)
(3, 647)
(3, 648)
(3, 649)
(3, 650)
(3, 651)
(3, 652)
(3, 653)
(3, 654)
(3, 655)
(3, 656)
(3, 657)
(3, 658)
(3, 659)
(3, 660)
(3, 661)
(3, 662)
(3, 663)
(3, 664)
(3, 665)
(3, 666)
(3, 667)
(3, 668)
(3, 669)
(3, 670)
(3, 671)
(3, 672)
(3, 673)
(3, 674)
(3, 675)
(3, 676)
(3, 677)
(3, 678)
(3, 679)
(3, 680)
(3, 681)
(3, 682)
(3, 683)
(3, 684)
(3, 685)
(3, 686)
(3, 687)
(3, 688)
(3, 689)
(3, 690)
(3, 691)
(3, 692)
(3, 693)
(3, 694)
(3, 695)
(3, 696)
(3, 697)
(3, 698)
(3, 699)
(3, 700)
(3, 701)
(3, 702)
(3, 703)
(3, 704)
(3, 705)
(3, 706)
(3, 707)
(3, 708)
(3, 709)
(3, 710)
(3, 711)
(3, 712)
(3, 713)
(3, 714)
(3, 715)
(3, 716)
(3, 717)
(3, 718)
(3, 719)
(3, 720)
(3, 721)
(3, 722)
(3, 723)
(3, 724)
(3, 725)
(3, 726)
(3, 727)
(3, 728)
(3, 729)
(3, 730)
(3, 731)
(3, 732)
(3, 733)
(3, 734)
(3, 735)
(3, 736)
(3, 737)
(3, 738)
(3, 739)
(3, 740)
(3, 741)
(3, 742)
(3, 743)
(3, 744)
(3, 745)
(3, 746)
(3, 747)
(3, 748)
(3, 749)
(3, 750)
(3, 751)
(3, 752)
(3, 753)
(

(5, 643)
(5, 644)
(5, 645)
(5, 646)
(5, 647)
(5, 648)
(5, 649)
(5, 650)
(5, 651)
(5, 652)
(5, 653)
(5, 654)
(5, 655)
(5, 656)
(5, 657)
(5, 658)
(5, 659)
(5, 660)
(5, 661)
(5, 662)
(5, 663)
(5, 664)
(5, 665)
(5, 666)
(5, 667)
(5, 668)
(5, 669)
(5, 670)
(5, 671)
(5, 672)
(5, 673)
(5, 674)
(5, 675)
(5, 676)
(5, 677)
(5, 678)
(5, 679)
(5, 680)
(5, 681)
(5, 682)
(5, 683)
(5, 684)
(5, 685)
(5, 686)
(5, 687)
(5, 688)
(5, 689)
(5, 690)
(5, 691)
(5, 692)
(5, 693)
(5, 694)
(5, 695)
(5, 696)
(5, 697)
(5, 698)
(5, 699)
(5, 700)
(5, 701)
(5, 702)
(5, 703)
(5, 704)
(5, 705)
(5, 706)
(5, 707)
(5, 708)
(5, 709)
(5, 710)
(5, 711)
(5, 712)
(5, 713)
(5, 714)
(5, 715)
(5, 716)
(5, 717)
(5, 718)
(5, 719)
(5, 720)
(5, 721)
(5, 722)
(5, 723)
(5, 724)
(5, 725)
(5, 726)
(5, 727)
(5, 728)
(5, 729)
(5, 730)
(5, 731)
(5, 732)
(5, 733)
(5, 734)
(5, 735)
(5, 736)
(5, 737)
(5, 738)
(5, 739)
(5, 740)
(5, 741)
(5, 742)
(5, 743)
(5, 744)
(5, 745)
(5, 746)
(5, 747)
(5, 748)
(5, 749)
(5, 750)
(5, 751)
(5, 752)
(5, 753)
(

(7, 642)
(7, 643)
(7, 644)
(7, 645)
(7, 646)
(7, 647)
(7, 648)
(7, 649)
(7, 650)
(7, 651)
(7, 652)
(7, 653)
(7, 654)
(7, 655)
(7, 656)
(7, 657)
(7, 658)
(7, 659)
(7, 660)
(7, 661)
(7, 662)
(7, 663)
(7, 664)
(7, 665)
(7, 666)
(7, 667)
(7, 668)
(7, 669)
(7, 670)
(7, 671)
(7, 672)
(7, 673)
(7, 674)
(7, 675)
(7, 676)
(7, 677)
(7, 678)
(7, 679)
(7, 680)
(7, 681)
(7, 682)
(7, 683)
(7, 684)
(7, 685)
(7, 686)
(7, 687)
(7, 688)
(7, 689)
(7, 690)
(7, 691)
(7, 692)
(7, 693)
(7, 694)
(7, 695)
(7, 696)
(7, 697)
(7, 698)
(7, 699)
(7, 700)
(7, 701)
(7, 702)
(7, 703)
(7, 704)
(7, 705)
(7, 706)
(7, 707)
(7, 708)
(7, 709)
(7, 710)
(7, 711)
(7, 712)
(7, 713)
(7, 714)
(7, 715)
(7, 716)
(7, 717)
(7, 718)
(7, 719)
(7, 720)
(7, 721)
(7, 722)
(7, 723)
(7, 724)
(7, 725)
(7, 726)
(7, 727)
(7, 728)
(7, 729)
(7, 730)
(7, 731)
(7, 732)
(7, 733)
(7, 734)
(7, 735)
(7, 736)
(7, 737)
(7, 738)
(7, 739)
(7, 740)
(7, 741)
(7, 742)
(7, 743)
(7, 744)
(7, 745)
(7, 746)
(7, 747)
(7, 748)
(7, 749)
(7, 750)
(7, 751)
(7, 752)
(

...

(17, 640)
(17, 641)
(17, 642)
(17, 643)
(17, 644)
(17, 645)
(17, 646)
(17, 647)
(17, 648)
(17, 649)
(17, 650)
(17, 651)
(17, 652)
(17, 653)
(17, 654)
(17, 655)
(17, 656)
(17, 657)
(17, 658)
(17, 659)
(17, 660)
(17, 661)
(17, 662)
(17, 663)
(17, 664)
(17, 665)
(17, 666)
(17, 667)
(17, 668)
(17, 669)
(17, 670)
(17, 671)
(17, 672)
(17, 673)
(17, 674)
(17, 675)
(17, 676)
(17, 677)
(17, 678)
(17, 679)
(17, 680)
(17, 681)
(17, 682)
(17, 683)
(17, 684)
(17, 685)
(17, 686)
(17, 687)
(17, 688)
(17, 689)
(17, 690)
(17, 691)
(17, 692)
(17, 693)
(17, 694)
(17, 695)
(17, 696)
(17, 697)
(17, 698)
(17, 699)
(17, 700)
(17, 701)
(17, 702)
(17, 703)
(17, 704)
(17, 705)
(17, 706)
(17, 707)
(17, 708)
(17, 709)
(17, 710)
(17, 711)
(17, 712)
(17, 713)
(17, 714)
(17, 715)
(17, 716)
(17, 717)
(17, 718)
(17, 719)
(17, 720)
(17, 721)
(17, 722)
(17, 723)
(17, 724)
(17, 725)
(17, 726)
(17, 727)
(17, 728)
(17, 729)
(17, 730)
(17, 731)
(17, 732)
(17, 733)
(17, 734)
(17, 735)
(17, 736)
(17, 737)
(17, 738)
(17, 739)


(19, 639)
(19, 640)
(19, 641)
(19, 642)
(19, 643)
(19, 644)
(19, 645)
(19, 646)
(19, 647)
(19, 648)
(19, 649)
(19, 650)
(19, 651)
(19, 652)
(19, 653)
(19, 654)
(19, 655)
(19, 656)
(19, 657)
(19, 658)
(19, 659)
(19, 660)
(19, 661)
(19, 662)
(19, 663)
(19, 664)
(19, 665)
(19, 666)
(19, 667)
(19, 668)
(19, 669)
(19, 670)
(19, 671)
(19, 672)
(19, 673)
(19, 674)
(19, 675)
(19, 676)
(19, 677)
(19, 678)
(19, 679)
(19, 680)
(19, 681)
(19, 682)
(19, 683)
(19, 684)
(19, 685)
(19, 686)
(19, 687)
(19, 688)
(19, 689)
(19, 690)
(19, 691)
(19, 692)
(19, 693)
(19, 694)
(19, 695)
(19, 696)
(19, 697)
(19, 698)
(19, 699)
(19, 700)
(19, 701)
(19, 702)
(19, 703)
(19, 704)
(19, 705)
(19, 706)
(19, 707)
(19, 708)
(19, 709)
(19, 710)
(19, 711)
(19, 712)
(19, 713)
(19, 714)
(19, 715)
(19, 716)
(19, 717)
(19, 718)
(19, 719)
(19, 720)
(19, 721)
(19, 722)
(19, 723)
(19, 724)
(19, 725)
(19, 726)
(19, 727)
(19, 728)
(19, 729)
(19, 730)
(19, 731)
(19, 732)
(19, 733)
(19, 734)
(19, 735)
(19, 736)
(19, 737)
(19, 738)


(21, 639)
(21, 640)
(21, 641)
(21, 642)
(21, 643)
(21, 644)
(21, 645)
(21, 646)
(21, 647)
(21, 648)
(21, 649)
(21, 650)
(21, 651)
(21, 652)
(21, 653)
(21, 654)
(21, 655)
(21, 656)
(21, 657)
(21, 658)
(21, 659)
(21, 660)
(21, 661)
(21, 662)
(21, 663)
(21, 664)
(21, 665)
(21, 666)
(21, 667)
(21, 668)
(21, 669)
(21, 670)
(21, 671)
(21, 672)
(21, 673)
(21, 674)
(21, 675)
(21, 676)
(21, 677)
(21, 678)
(21, 679)
(21, 680)
(21, 681)
(21, 682)
(21, 683)
(21, 684)
(21, 685)
(21, 686)
(21, 687)
(21, 688)
(21, 689)
(21, 690)
(21, 691)
(21, 692)
(21, 693)
(21, 694)
(21, 695)
(21, 696)
(21, 697)
(21, 698)
(21, 699)
(21, 700)
(21, 701)
(21, 702)
(21, 703)
(21, 704)
(21, 705)
(21, 706)
(21, 707)
(21, 708)
(21, 709)
(21, 710)
(21, 711)
(21, 712)
(21, 713)
(21, 714)
(21, 715)
(21, 716)
(21, 717)
(21, 718)
(21, 719)
(21, 720)
(21, 721)
(21, 722)
(21, 723)
(21, 724)
(21, 725)
(21, 726)
(21, 727)
(21, 728)
(21, 729)
(21, 730)
(21, 731)
(21, 732)
(21, 733)
(21, 734)
(21, 735)
(21, 736)
(21, 737)
(21, 738)


(23, 638)
(23, 639)
(23, 640)
(23, 641)
(23, 642)
(23, 643)
(23, 644)
(23, 645)
(23, 646)
(23, 647)
(23, 648)
(23, 649)
(23, 650)
(23, 651)
(23, 652)
(23, 653)
(23, 654)
(23, 655)
(23, 656)
(23, 657)
(23, 658)
(23, 659)
(23, 660)
(23, 661)
(23, 662)
(23, 663)
(23, 664)
(23, 665)
(23, 666)
(23, 667)
(23, 668)
(23, 669)
(23, 670)
(23, 671)
(23, 672)
(23, 673)
(23, 674)
(23, 675)
(23, 676)
(23, 677)
(23, 678)
(23, 679)
(23, 680)
(23, 681)
(23, 682)
(23, 683)
(23, 684)
(23, 685)
(23, 686)
(23, 687)
(23, 688)
(23, 689)
(23, 690)
(23, 691)
(23, 692)
(23, 693)
(23, 694)
(23, 695)
(23, 696)
(23, 697)
(23, 698)
(23, 699)
(23, 700)
(23, 701)
(23, 702)
(23, 703)
(23, 704)
(23, 705)
(23, 706)
(23, 707)
(23, 708)
(23, 709)
(23, 710)
(23, 711)
(23, 712)
(23, 713)
(23, 714)
(23, 715)
(23, 716)
(23, 717)
(23, 718)
(23, 719)
(23, 720)
(23, 721)
(23, 722)
(23, 723)
(23, 724)
(23, 725)
(23, 726)
(23, 727)
(23, 728)
(23, 729)
(23, 730)
(23, 731)
(23, 732)
(23, 733)
(23, 734)
(23, 735)
(23, 736)
(23, 737)


KeyboardInterrupt: 

In [76]:
# Start timer.
import time
t0 = time.time()

# Import file.
file = open('./day_3_input.txt', 'r')

# Instantiate overlap counter.
overlap_counter = 0

# Instantiate pixel_dict dictionary.
pixel_dict = {(i,j): 0 for i in range(1000) for j in range(1000)}

# Iterate through each key in pixel_dict.
for pixel in pixel_dict.keys():
    
    # Check to make sure we're making progress. 
    if pixel[1] == 0 and pixel[0] % 25 == 0:
        print(f'We are on row {pixel[0]} of 1,000 after {round(time.time()-t0,1)} seconds.')
    
    # Iterate through each item in our input.
    for i in file:        
        
        # Transform our input into coordinates.
        coords = transform(i)
        
        # Check if the pixel is in the columns to be cut.
        if pixel[0] >= coords[0]:
            if pixel[0] <= coords[2]:
                
                # Check if the pixel is in the rows to be cut.
                if pixel[1] >= coords[1]:
                    if pixel[1] <= coords[3]:
                        
                        # If the pixel is in the columns and
                        # rows to be cut, add one to count.
                        pixel_dict[pixel] += 1
                        
                        # If our pixel is already covered by 2 squares,
                        # then add one to overlap_counter, then break
                        # out and move onto the next pixel. This isn't 
                        # necessary, but should hopefully speed things up!
                        if pixel_dict[pixel] >= 2:
                            overlap_counter += 1
                            break

maximum = 0
for v in pixel_dict.values():
    if v > maximum:
        maximum = v

print(maximum)
print(overlap_counter)

We are on row 0 of 1,000 after 0.4 seconds.
We are on row 25 of 1,000 after 0.5 seconds.
We are on row 50 of 1,000 after 0.6 seconds.
We are on row 75 of 1,000 after 0.8 seconds.
We are on row 100 of 1,000 after 0.9 seconds.
We are on row 125 of 1,000 after 1.0 seconds.
We are on row 150 of 1,000 after 1.1 seconds.
We are on row 175 of 1,000 after 1.3 seconds.
We are on row 200 of 1,000 after 1.4 seconds.
We are on row 225 of 1,000 after 1.5 seconds.
We are on row 250 of 1,000 after 1.6 seconds.
We are on row 275 of 1,000 after 1.7 seconds.
We are on row 300 of 1,000 after 1.8 seconds.
We are on row 325 of 1,000 after 2.0 seconds.
We are on row 350 of 1,000 after 2.1 seconds.
We are on row 375 of 1,000 after 2.2 seconds.
We are on row 400 of 1,000 after 2.3 seconds.
We are on row 425 of 1,000 after 2.5 seconds.
We are on row 450 of 1,000 after 2.6 seconds.
We are on row 475 of 1,000 after 2.7 seconds.
We are on row 500 of 1,000 after 2.8 seconds.
We are on row 525 of 1,000 after 2.9 se

How is it that we aren't getting values?
- Hypothesis: I'm using `i` multiple times; maybe this is causing issues?

In [77]:
# Start timer.
import time
t0 = time.time()

# Import file.
file = open('./day_3_input.txt', 'r')

# Instantiate overlap counter.
overlap_counter = 0

# Instantiate pixel_dict dictionary.
pixel_dict = {(i,j): 0 for i in range(1000) for j in range(1000)}

# Iterate through each item in pixel_dict.
for pixel in pixel_dict.keys():
    
    # Check to make sure we're making progress. 
    if pixel[1] == 0 and pixel[0] % 25 == 0:
        print(f'We are on row {pixel[0]} of 1,000 after {round(time.time()-t0,1)} seconds.')
    
    # Iterate through each item in our input.
    for code in file:        
        
        # Transform our input into coordinates.
        coords = transform(code)
        
        # Check if the pixel is in the columns to be cut.
        if pixel[0] >= coords[0]:
            if pixel[0] <= coords[2]:
                
                # Check if the pixel is in the rows to be cut.
                if pixel[1] >= coords[1]:
                    if pixel[1] <= coords[3]:
                        
                        # If the pixel is in the columns and
                        # rows to be cut, add one to count.
                        pixel_dict[pixel] += 1
                        
                        # If our pixel is already covered by 2 squares,
                        # then add one to overlap_counter, then break
                        # out and move onto the next pixel. This isn't 
                        # necessary, but should hopefully speed things up!
                        if pixel_dict[pixel] >= 2:
                            overlap_counter += 1
                            break

maximum = 0
for v in pixel_dict.values():
    if v > maximum:
        maximum = v

print(maximum)
print(overlap_counter)

We are on row 0 of 1,000 after 0.4 seconds.
We are on row 25 of 1,000 after 0.5 seconds.
We are on row 50 of 1,000 after 0.6 seconds.
We are on row 75 of 1,000 after 0.8 seconds.
We are on row 100 of 1,000 after 0.9 seconds.
We are on row 125 of 1,000 after 1.0 seconds.
We are on row 150 of 1,000 after 1.2 seconds.
We are on row 175 of 1,000 after 1.3 seconds.
We are on row 200 of 1,000 after 1.4 seconds.
We are on row 225 of 1,000 after 1.5 seconds.
We are on row 250 of 1,000 after 1.6 seconds.
We are on row 275 of 1,000 after 1.7 seconds.
We are on row 300 of 1,000 after 1.9 seconds.
We are on row 325 of 1,000 after 2.0 seconds.
We are on row 350 of 1,000 after 2.2 seconds.
We are on row 375 of 1,000 after 2.3 seconds.
We are on row 400 of 1,000 after 2.4 seconds.
We are on row 425 of 1,000 after 2.5 seconds.
We are on row 450 of 1,000 after 2.6 seconds.
We are on row 475 of 1,000 after 2.8 seconds.
We are on row 500 of 1,000 after 2.9 seconds.
We are on row 525 of 1,000 after 3.0 se

Nope. That isn't it. I wonder if the nested `if` statements aren't getting triggered. Let's add an output to see if they ever do get met.

In [79]:
# Start timer.
import time
t0 = time.time()

# Import file.
file = open('./day_3_input.txt', 'r')

# Instantiate overlap counter.
overlap_counter = 0

# Instantiate pixel_dict dictionary.
pixel_dict = {(i,j): 0 for i in range(1000) for j in range(1000)}

# Iterate through each item in pixel_dict.
for pixel in pixel_dict.keys():
    
    # Check to make sure we're making progress. 
#     if pixel[1] == 0 and pixel[0] % 25 == 0:
#         print(f'We are on row {pixel[0]} of 1,000 after {round(time.time()-t0,1)} seconds.')
    
    # Iterate through each item in our input.
    for code in file:        
        
        # Transform our input into coordinates.
        coords = transform(code)
        
        # Check if the pixel is in the columns to be cut.
        if pixel[0] >= coords[0]:
            if pixel[0] <= coords[2]:
                
                # Check if the pixel is in the rows to be cut.
                if pixel[1] >= coords[1]:
                    if pixel[1] <= coords[3]:
                        
                        # If the pixel is in the columns and
                        # rows to be cut, add one to count.
                        print("We should be adding 1 here!")
                        pixel_dict[pixel] += 1
                        
                        # If our pixel is already covered by 2 squares,
                        # then add one to overlap_counter, then break
                        # out and move onto the next pixel. This isn't 
                        # necessary, but should hopefully speed things up!
                        if pixel_dict[pixel] >= 2:
                            overlap_counter += 1
                            break

maximum = 0
for v in pixel_dict.values():
    if v > maximum:
        maximum = v

print(maximum)
print(overlap_counter)

0
0


So, we're just never making it into that last step. Let's move our print statement out to see if we're making it into the third step.

In [80]:
# Start timer.
import time
t0 = time.time()

# Import file.
file = open('./day_3_input.txt', 'r')

# Instantiate overlap counter.
overlap_counter = 0

# Instantiate pixel_dict dictionary.
pixel_dict = {(i,j): 0 for i in range(1000) for j in range(1000)}

# Iterate through each item in pixel_dict.
for pixel in pixel_dict.keys():
    
    # Check to make sure we're making progress. 
#     if pixel[1] == 0 and pixel[0] % 25 == 0:
#         print(f'We are on row {pixel[0]} of 1,000 after {round(time.time()-t0,1)} seconds.')
    
    # Iterate through each item in our input.
    for code in file:        
        
        # Transform our input into coordinates.
        coords = transform(code)
        
        # Check if the pixel is in the columns to be cut.
        if pixel[0] >= coords[0]:
            if pixel[0] <= coords[2]:
                
                # Check if the pixel is in the rows to be cut.
                if pixel[1] >= coords[1]:
                    print("We made it to the third step!")
                    if pixel[1] <= coords[3]:
                        
                        # If the pixel is in the columns and
                        # rows to be cut, add one to count.
                        pixel_dict[pixel] += 1
                        
                        # If our pixel is already covered by 2 squares,
                        # then add one to overlap_counter, then break
                        # out and move onto the next pixel. This isn't 
                        # necessary, but should hopefully speed things up!
                        if pixel_dict[pixel] >= 2:
                            overlap_counter += 1
                            break

maximum = 0
for v in pixel_dict.values():
    if v > maximum:
        maximum = v

print(maximum)
print(overlap_counter)

0
0


And the second...

In [81]:
# Start timer.
import time
t0 = time.time()

# Import file.
file = open('./day_3_input.txt', 'r')

# Instantiate overlap counter.
overlap_counter = 0

# Instantiate pixel_dict dictionary.
pixel_dict = {(i,j): 0 for i in range(1000) for j in range(1000)}

# Iterate through each item in pixel_dict.
for pixel in pixel_dict.keys():
    
    # Check to make sure we're making progress. 
#     if pixel[1] == 0 and pixel[0] % 25 == 0:
#         print(f'We are on row {pixel[0]} of 1,000 after {round(time.time()-t0,1)} seconds.')
    
    # Iterate through each item in our input.
    for code in file:        
        
        # Transform our input into coordinates.
        coords = transform(code)
        
        # Check if the pixel is in the columns to be cut.
        if pixel[0] >= coords[0]:
            if pixel[0] <= coords[2]:
                print("We made it to the second step!")
                
                # Check if the pixel is in the rows to be cut.
                if pixel[1] >= coords[1]:
                    if pixel[1] <= coords[3]:
                        
                        # If the pixel is in the columns and
                        # rows to be cut, add one to count.
                        pixel_dict[pixel] += 1
                        
                        # If our pixel is already covered by 2 squares,
                        # then add one to overlap_counter, then break
                        # out and move onto the next pixel. This isn't 
                        # necessary, but should hopefully speed things up!
                        if pixel_dict[pixel] >= 2:
                            overlap_counter += 1
                            break

maximum = 0
for v in pixel_dict.values():
    if v > maximum:
        maximum = v

print(maximum)
print(overlap_counter)

We made it to the second step!
0
0


This is odd. How is it that we only make it to the second step once? Something is not right here.
- It could be that I incorrectly defined my `transform` function above?

After checking that out, it doesn't look like it's incorrectly defined.

Am I comparing the wrong parts of `pixel` and `coords`?

In [94]:
pixel = (6, 4)

# Transform our input into coordinates.
coords = transform('#123 @ 3,2: 5x4')

# Check if the pixel is in the columns to be cut.
if pixel[0] >= coords[0]:
    if pixel[0] <= coords[2]:

        # Check if the pixel is in the rows to be cut.
        if pixel[1] >= coords[1]:
            if pixel[1] <= coords[3]:

                # If the pixel is in the columns and
                # rows to be cut, add one to count.
                pixel_dict[pixel] += 1

                # If our pixel is already covered by 2 squares,
                # then add one to overlap_counter, then break
                # out and move onto the next pixel. This isn't 
                # necessary, but should hopefully speed things up!
                if pixel_dict[pixel] == 2:
                    overlap_counter += 1

In [95]:
pixel_dict[(6,4)]

5

In [96]:
overlap_counter

1

That doesn't look like it either!

Maybe `pixel` doesn't take on the value I expect it to take on.

In [123]:
# Start timer.
import time
t0 = time.time()

# Import file.
file = open('./day_3_input.txt', 'r')

# Instantiate overlap counter.
overlap_counter = 0

# Instantiate pixel_dict dictionary.
pixel_dict = {(i,j): 0 for i in range(1000) for j in range(1000)}

# Iterate through each item in pixel_dict.
for pixel in pixel_dict.keys():
        
    # Iterate through each item in our input.
    for code in file:        
        
        # Transform our input into coordinates.
        coords = transform(code)
        
        # Check if the pixel is in the columns to be cut.
        if pixel[0] >= coords[0]:
            if pixel[0] <= coords[2]:
                print("We made it to the second step!")
                print(pixel, coords)
                
                # Check if the pixel is in the rows to be cut.
                if pixel[1] >= coords[1]:
                    if pixel[1] <= coords[3]:
                        
                        # If the pixel is in the columns and
                        # rows to be cut, add one to count.
                        pixel_dict[pixel] += 1
                        
                        # If our pixel is already covered by 2 squares,
                        # then add one to overlap_counter, then break
                        # out and move onto the next pixel. This isn't 
                        # necessary, but should hopefully speed things up!
                        if pixel_dict[pixel] >= 2:
                            overlap_counter += 1
                            break
                            
    # Check to make sure we're making progress. 
    if pixel[1] == 0 and pixel[0] % 25 == 0:
        print(f'We finished row {pixel[0]} of 1,000 after {round(time.time()-t0,1)} seconds.')

maximum = 0
for v in pixel_dict.values():
    if v > maximum:
        maximum = v

print(maximum)
print(overlap_counter)

We made it to the second step!
(0, 0) (0, 778, 12, 800)
We finished row 0 of 1,000 after 0.4 seconds.
We finished row 25 of 1,000 after 0.5 seconds.
We finished row 50 of 1,000 after 0.7 seconds.
We finished row 75 of 1,000 after 0.8 seconds.
We finished row 100 of 1,000 after 0.9 seconds.
We finished row 125 of 1,000 after 1.0 seconds.
We finished row 150 of 1,000 after 1.1 seconds.
We finished row 175 of 1,000 after 1.3 seconds.
We finished row 200 of 1,000 after 1.4 seconds.
We finished row 225 of 1,000 after 1.5 seconds.
We finished row 250 of 1,000 after 1.6 seconds.
We finished row 275 of 1,000 after 1.7 seconds.
We finished row 300 of 1,000 after 1.9 seconds.
We finished row 325 of 1,000 after 2.0 seconds.
We finished row 350 of 1,000 after 2.1 seconds.
We finished row 375 of 1,000 after 2.2 seconds.
We finished row 400 of 1,000 after 2.3 seconds.
We finished row 425 of 1,000 after 2.5 seconds.
We finished row 450 of 1,000 after 2.6 seconds.
We finished row 475 of 1,000 after 2.

We're still not getting anything positive... which is weird.

In [117]:
for pixel in pixel_dict.keys():
    if pixel[0] % 100 == 0 and pixel[1] % 100 == 0:
        print(pixel[1])
    if pixel[0] == 300 and pixel[1] == 300:
        break

0
100
200
300
400
500
600
700
800
900
0
100
200
300
400
500
600
700
800
900
0
100
200
300
400
500
600
700
800
900
0
100
200
300


`pixel` is iterating as it should... did I screw up the nested `if` statements?

It looks like that isn't it, either. I'm going to swap my pixel and code iterations. Instead of searching each pixel first, I'll try each different code fragment.

In [130]:
# Start timer.
import time
t0 = time.time()

# Import file.
file = open('./day_3_input.txt', 'r')

# Instantiate overlap counter.
overlap_counter = 0

# Instantiate pixel_dict dictionary.
pixel_dict = {(i,j): 0 for i in range(1000) for j in range(1000)}

# Instantiate code counter to track how quickly I'm going.
code_count = 0

# Iterate through each item in our input.
for code in file:
    
    # Transform our input into coordinates.
    coords = transform(code)

    # Iterate through each item in pixel_dict.
    for pixel in pixel_dict.keys():
    
        # Check if the pixel is in the columns to be cut.
        if pixel[0] >= coords[0]:
            if pixel[0] <= coords[2]:
                if pixel[1] >= coords[1]:
                    if pixel[1] <= coords[3]:
                        
                        # If the pixel is in the columns and
                        # rows to be cut, add one to count.
                        pixel_dict[pixel] += 1

                        # If our pixel is already covered by 2 squares,
                        # then add one to overlap_counter, then break
                        # out and move onto the next pixel. This isn't 
                        # necessary, but should hopefully speed things up!
                        if pixel_dict[pixel] >= 2:
                            overlap_counter += 1
                            break
    
    code_count += 1

    # Check to make sure we're making progress. 
    if code_count % 25 == 0:
        print(f'We finished claim {code_count} of 1,233 after {round(time.time()-t0,1)} seconds.')

maximum = 0
for v in pixel_dict.values():
    if v > maximum:
        maximum = v

print(maximum)
print(overlap_counter)

We finished claim 25 of 1,233 after 4.2 seconds.
We finished claim 50 of 1,233 after 7.7 seconds.
We finished claim 75 of 1,233 after 11.3 seconds.
We finished claim 100 of 1,233 after 14.9 seconds.
We finished claim 125 of 1,233 after 18.9 seconds.
We finished claim 150 of 1,233 after 22.4 seconds.
We finished claim 175 of 1,233 after 25.5 seconds.
We finished claim 200 of 1,233 after 28.7 seconds.
We finished claim 225 of 1,233 after 31.9 seconds.
We finished claim 250 of 1,233 after 34.6 seconds.
We finished claim 275 of 1,233 after 37.3 seconds.
We finished claim 300 of 1,233 after 40.1 seconds.
We finished claim 325 of 1,233 after 42.9 seconds.
We finished claim 350 of 1,233 after 45.5 seconds.
We finished claim 375 of 1,233 after 48.6 seconds.
We finished claim 400 of 1,233 after 51.4 seconds.
We finished claim 425 of 1,233 after 53.5 seconds.
We finished claim 450 of 1,233 after 55.7 seconds.
We finished claim 475 of 1,233 after 58.0 seconds.
We finished claim 500 of 1,233 after

**Okay!** Instead of looping through pixels, then code, I swapped them so I iterate through code, then pixels. I'm not entirely certain at this point *why* this is giving me substantially different results, but I'm in the weeds and it's hard to step back and look at the full picture. My answer of 787 isn't right (too low), but at least I'm getting non-zero answers!

Let's see if, by swapping the loops, I screwed things up later in my code.
- By using a `break` down near the bottom, I'm breaking out of the wrong loop!

In [131]:
# Start timer.
import time
t0 = time.time()

# Import file.
file = open('./day_3_input.txt', 'r')

# Instantiate overlap counter.
overlap_counter = 0

# Instantiate pixel_dict dictionary.
pixel_dict = {(i,j): 0 for i in range(1000) for j in range(1000)}

# Instantiate code counter to track how quickly I'm going.
code_count = 0

# Iterate through each item in our input.
for code in file:
    
    # Transform our input into coordinates.
    coords = transform(code)

    # Iterate through each item in pixel_dict.
    for pixel in pixel_dict.keys():
    
        # Check if the pixel is in the columns to be cut.
        if pixel[0] >= coords[0]:
            if pixel[0] <= coords[2]:
                if pixel[1] >= coords[1]:
                    if pixel[1] <= coords[3]:
                        
                        # If the pixel is in the columns and
                        # rows to be cut, add one to count.
                        pixel_dict[pixel] += 1

                        # If our pixel is already covered by 2 squares,
                        # then add one to overlap_counter, then break
                        # out and move onto the next pixel. This isn't 
                        # necessary, but should hopefully speed things up!
#                         if pixel_dict[pixel] >= 2:
#                             overlap_counter += 1
#                             break
    
    code_count += 1

    # Check to make sure we're making progress. 
    if code_count % 25 == 0:
        print(f'We finished claim {code_count} of 1,233 after {round(time.time()-t0,1)} seconds.')

maximum = 0
for v in pixel_dict.values():
    if v > maximum:
        maximum = v

print(maximum)
print(overlap_counter)

We finished claim 25 of 1,233 after 4.3 seconds.
We finished claim 50 of 1,233 after 8.2 seconds.
We finished claim 75 of 1,233 after 11.9 seconds.
We finished claim 100 of 1,233 after 15.8 seconds.
We finished claim 125 of 1,233 after 19.9 seconds.
We finished claim 150 of 1,233 after 24.2 seconds.
We finished claim 175 of 1,233 after 28.5 seconds.
We finished claim 200 of 1,233 after 32.2 seconds.
We finished claim 225 of 1,233 after 36.2 seconds.
We finished claim 250 of 1,233 after 39.7 seconds.
We finished claim 275 of 1,233 after 43.5 seconds.
We finished claim 300 of 1,233 after 47.2 seconds.
We finished claim 325 of 1,233 after 50.9 seconds.
We finished claim 350 of 1,233 after 54.7 seconds.
We finished claim 375 of 1,233 after 58.6 seconds.
We finished claim 400 of 1,233 after 62.2 seconds.
We finished claim 425 of 1,233 after 65.9 seconds.
We finished claim 450 of 1,233 after 69.9 seconds.
We finished claim 475 of 1,233 after 73.5 seconds.
We finished claim 500 of 1,233 after

So *I think* things are moving in the right direction... but in my haste, while debugging something, I commented out the counter that was tracking the thing I want to measure!

In [132]:
# Start timer.
import time
t0 = time.time()

# Import file.
file = open('./day_3_input.txt', 'r')

# Instantiate overlap counter.
overlap_counter = 0

# Instantiate pixel_dict dictionary.
pixel_dict = {(i,j): 0 for i in range(1000) for j in range(1000)}

# Instantiate code counter to track how quickly I'm going.
code_count = 0

# Iterate through each item in our input.
for code in file:
    
    # Transform our input into coordinates.
    coords = transform(code)

    # Iterate through each item in pixel_dict.
    for pixel in pixel_dict.keys():
    
        # Check if the pixel is in the columns to be cut.
        if pixel[0] >= coords[0]:
            if pixel[0] <= coords[2]:
                if pixel[1] >= coords[1]:
                    if pixel[1] <= coords[3]:
                        
                        # If the pixel is in the columns and
                        # rows to be cut, add one to count.
                        pixel_dict[pixel] += 1

    code_count += 1

    # Check to make sure we're making progress. 
    if code_count % 25 == 0:
        print(f'We finished claim {code_count} of 1,233 after {round(time.time()-t0,1)} seconds.')
        
for pixel in pixel_dict.keys():
    if pixel_dict[pixel] >= 2:
        overlap_counter += 1

maximum = 0
for v in pixel_dict.values():
    if v > maximum:
        maximum = v

print(maximum)
print(overlap_counter)

We finished claim 25 of 1,233 after 4.2 seconds.
We finished claim 50 of 1,233 after 7.9 seconds.
We finished claim 75 of 1,233 after 11.6 seconds.
We finished claim 100 of 1,233 after 15.1 seconds.
We finished claim 125 of 1,233 after 19.0 seconds.
We finished claim 150 of 1,233 after 22.8 seconds.
We finished claim 175 of 1,233 after 26.5 seconds.
We finished claim 200 of 1,233 after 30.2 seconds.
We finished claim 225 of 1,233 after 34.2 seconds.
We finished claim 250 of 1,233 after 37.7 seconds.
We finished claim 275 of 1,233 after 41.5 seconds.
We finished claim 300 of 1,233 after 45.1 seconds.
We finished claim 325 of 1,233 after 48.8 seconds.
We finished claim 350 of 1,233 after 52.6 seconds.
We finished claim 375 of 1,233 after 56.5 seconds.
We finished claim 400 of 1,233 after 60.1 seconds.
We finished claim 425 of 1,233 after 63.8 seconds.
We finished claim 450 of 1,233 after 67.5 seconds.
We finished claim 475 of 1,233 after 71.1 seconds.
We finished claim 500 of 1,233 after

### Part Two

#### My process: 
- I will loop through each code snippet and see if a corner of my first code snippet is inside any other code snippet. (Update: this turned out to be wrong.)

In [242]:
# Start timer.
import time
t0 = time.time()

# Import file.
file = open('./day_3_input.txt', 'r')

file_list = [code for code in file]

lst = []

# Iterate through file.
for i in range(len(file_list)):
    
    # Retrieve coordinates of i.
    i_coords = transform(file_list[i])
    
# (i_coords[0], i_coords[1]) is the upper-left corner.
# (i_coords[0], i_coords[3]) is the lower-left corner.
# (i_coords[2], i_coords[3]) is the lower-right corner.
# (i_coords[2], i_coords[1]) is the upper-right corner.
    
    # Iterate through remainder of file.
    for j in file_list[i+1:]:
        
        # Retrieve coordinates of j.
        j_coords = transform(j)
        
        # Check if upper-left corner of i is inside j.
        if i_coords[0] >= j_coords[0]:
            if i_coords[0] <= j_coords[2]:
                if i_coords[1] >= j_coords[1]:
                    if i_coords[1] <= j_coords[3]:
                        lst.append(i)
                        break
        
        # Check if lower-left corner of i is inside j.
        if i_coords[0] >= j_coords[0]:
            if i_coords[0] <= j_coords[2]:
                if i_coords[3] >= j_coords[1]:
                    if i_coords[3] <= j_coords[3]:
                        lst.append(i)
                        break
        
        # Check if upper-right corner of i is inside j.
        if i_coords[2] >= j_coords[0]:
            if i_coords[2] <= j_coords[2]:
                if i_coords[1] >= j_coords[1]:
                    if i_coords[1] <= j_coords[3]:
                        lst.append(i)
                        break
                        
        # Check if lower-right corner of i is inside j.
        if i_coords[2] >= j_coords[0]:
            if i_coords[2] <= j_coords[2]:
                if i_coords[3] >= j_coords[1]:
                    if i_coords[3] <= j_coords[3]:
                        lst.append(i)
                        break
                        
        if j == file_list[-1]:
            print(i)
    
    if i % 100 == 0:
        print(f'We are through iteration {i} of 1,233 after {round(time.time()-t0,3)} seconds.')
        
    if len(lst) != i + 1:
        print(f'The answer is {file_list[i]}.')

We are through iteration 0 of 1,233 after 0.004 seconds.
15
The answer is #16 @ 859,624: 21x19
.
The answer is #17 @ 869,902: 16x29
.
The answer is #18 @ 126,834: 24x17
.
The answer is #19 @ 873,627: 18x15
.
The answer is #20 @ 853,240: 18x28
.
The answer is #21 @ 863,830: 18x11
.
The answer is #22 @ 54,732: 20x23
.
The answer is #23 @ 253,825: 20x10
.
The answer is #24 @ 261,710: 14x13
.
The answer is #25 @ 383,456: 24x25
.
The answer is #26 @ 792,324: 28x19
.
The answer is #27 @ 163,336: 11x12
.
The answer is #28 @ 983,534: 14x11
.
28
The answer is #29 @ 879,249: 11x23
.
The answer is #30 @ 702,348: 21x28
.
The answer is #31 @ 730,51: 18x18
.
The answer is #32 @ 709,647: 26x26
.
The answer is #33 @ 850,90: 26x28
.
The answer is #34 @ 56,194: 14x14
.
The answer is #35 @ 285,967: 23x28
.
The answer is #36 @ 868,281: 27x22
.
The answer is #37 @ 243,692: 11x18
.
The answer is #38 @ 111,508: 15x20
.
The answer is #39 @ 851,414: 23x12
.
The answer is #40 @ 717,593: 10x24
.
The answer is #4

272
The answer is #273 @ 179,973: 25x20
.
The answer is #274 @ 68,445: 14x19
.
The answer is #275 @ 200,432: 26x11
.
The answer is #276 @ 713,521: 26x23
.
276
The answer is #277 @ 180,49: 15x22
.
The answer is #278 @ 753,862: 17x16
.
The answer is #279 @ 345,523: 13x27
.
The answer is #280 @ 547,22: 22x16
.
The answer is #281 @ 47,248: 11x29
.
The answer is #282 @ 816,132: 18x26
.
The answer is #283 @ 453,647: 15x28
.
The answer is #284 @ 175,907: 12x16
.
The answer is #285 @ 883,447: 18x18
.
The answer is #286 @ 642,733: 10x16
.
The answer is #287 @ 330,200: 10x12
.
The answer is #288 @ 672,331: 18x28
.
The answer is #289 @ 305,477: 13x28
.
The answer is #290 @ 840,853: 19x29
.
The answer is #291 @ 520,341: 25x10
.
The answer is #292 @ 32,142: 27x14
.
292
The answer is #293 @ 495,365: 21x18
.
The answer is #294 @ 920,906: 29x15
.
The answer is #295 @ 136,138: 23x17
.
The answer is #296 @ 520,222: 14x17
.
The answer is #297 @ 407,494: 27x23
.
The answer is #298 @ 801,847: 14x20
.
The a

The answer is #522 @ 72,781: 15x17
.
The answer is #523 @ 765,978: 12x12
.
The answer is #524 @ 267,565: 25x21
.
The answer is #525 @ 252,596: 16x16
.
The answer is #526 @ 9,867: 23x27
.
526
The answer is #527 @ 310,50: 24x26
.
The answer is #528 @ 34,841: 27x12
.
The answer is #529 @ 232,56: 26x11
.
The answer is #530 @ 979,45: 15x18
.
530
The answer is #531 @ 738,247: 17x29
.
The answer is #532 @ 607,341: 25x12
.
The answer is #533 @ 418,20: 27x24
.
The answer is #534 @ 926,56: 13x18
.
The answer is #535 @ 856,123: 23x27
.
The answer is #536 @ 612,299: 14x20
.
The answer is #537 @ 311,490: 21x19
.
537
The answer is #538 @ 408,826: 23x10
.
538
The answer is #539 @ 629,88: 14x16
.
The answer is #540 @ 908,733: 27x26
.
The answer is #541 @ 143,207: 21x15
.
The answer is #542 @ 303,545: 21x29
.
The answer is #543 @ 975,64: 13x28
.
The answer is #544 @ 932,373: 28x10
.
The answer is #545 @ 876,417: 10x10
.
545
The answer is #546 @ 193,767: 28x25
.
546
The answer is #547 @ 957,143: 13x10
.

.
732
The answer is #733 @ 741,361: 24x17
.
The answer is #734 @ 961,282: 18x21
.
The answer is #735 @ 62,533: 23x17
.
The answer is #736 @ 95,130: 23x12
.
The answer is #737 @ 230,581: 18x24
.
The answer is #738 @ 344,765: 12x18
.
The answer is #739 @ 78,204: 22x10
.
The answer is #740 @ 862,294: 13x18
.
The answer is #741 @ 131,949: 21x15
.
The answer is #742 @ 194,177: 23x24
.
742
The answer is #743 @ 106,68: 23x22
.
The answer is #744 @ 745,982: 28x17
.
The answer is #745 @ 366,202: 25x28
.
The answer is #746 @ 844,75: 16x11
.
746
The answer is #747 @ 765,281: 15x10
.
The answer is #748 @ 280,569: 26x16
.
The answer is #749 @ 84,458: 24x13
.
The answer is #750 @ 930,609: 23x11
.
The answer is #751 @ 642,189: 12x19
.
The answer is #752 @ 403,474: 24x25
.
The answer is #753 @ 489,246: 22x18
.
753
The answer is #754 @ 617,570: 25x29
.
754
The answer is #755 @ 491,557: 18x11
.
The answer is #756 @ 252,75: 22x18
.
The answer is #757 @ 938,833: 18x13
.
The answer is #758 @ 170,336: 24x22

961
The answer is #962 @ 518,545: 25x29
.
962
The answer is #963 @ 354,761: 15x16
.
The answer is #964 @ 709,777: 20x24
.
The answer is #965 @ 786,817: 28x14
.
965
The answer is #966 @ 105,129: 21x13
.
966
The answer is #967 @ 39,678: 4x17
.
The answer is #968 @ 594,621: 25x10
.
968
The answer is #969 @ 123,959: 22x16
.
969
The answer is #970 @ 479,298: 28x14
.
970
The answer is #971 @ 928,84: 12x17
.
The answer is #972 @ 924,955: 18x27
.
The answer is #973 @ 306,851: 13x16
.
The answer is #974 @ 288,830: 27x12
.
The answer is #975 @ 83,893: 29x18
.
The answer is #976 @ 877,112: 10x26
.
The answer is #977 @ 108,870: 19x17
.
977
The answer is #978 @ 370,97: 5x3
.
The answer is #979 @ 748,865: 23x20
.
979
The answer is #980 @ 271,92: 23x19
.
The answer is #981 @ 603,192: 25x28
.
981
The answer is #982 @ 860,522: 29x21
.
The answer is #983 @ 920,412: 19x27
.
983
The answer is #984 @ 473,806: 13x13
.
984
The answer is #985 @ 850,462: 14x10
.
The answer is #986 @ 593,184: 25x18
.
986
The an

In [241]:
pixel_dict[(340+26,139)]

1

I have been banging my head against the wall trying to figure this out. I checked each corner to see if it was inside any other rectangle. I thought that if all four corners of a piece of fabric weren't inside another piece of fabric, then we were good.

However, pieces of fabric could be entirely contained within one another or a rectangle could overlap on the border... so the corners is necessary, but not sufficient.
- This is a perfect situation where I thought a few steps ahead to be clever/speed things up... but I was just shooting myself in the foot.

Let's leverage this necessary piece to come up with a set of possible candidates, then we can check all values from `pixel_dict`.

In [244]:
# Start timer.
import time
t0 = time.time()

# Import file.
file = open('./day_3_input.txt', 'r')

file_list = [code for code in file]

lst = []

# Iterate through file.
for i in range(len(file_list)):
    
    # Retrieve coordinates of i.
    i_coords = transform(file_list[i])
    
# (i_coords[0], i_coords[1]) is the upper-left corner.
# (i_coords[0], i_coords[3]) is the lower-left corner.
# (i_coords[2], i_coords[3]) is the lower-right corner.
# (i_coords[2], i_coords[1]) is the upper-right corner.
    
    # Iterate through remainder of file.
    for j in file_list[i+1:]:
        
        # Retrieve coordinates of j.
        j_coords = transform(j)
        
        # Check if upper-left corner of i is inside j.
        if i_coords[0] >= j_coords[0]:
            if i_coords[0] <= j_coords[2]:
                if i_coords[1] >= j_coords[1]:
                    if i_coords[1] <= j_coords[3]:
                        break
        
        # Check if lower-left corner of i is inside j.
        if i_coords[0] >= j_coords[0]:
            if i_coords[0] <= j_coords[2]:
                if i_coords[3] >= j_coords[1]:
                    if i_coords[3] <= j_coords[3]:
                        break
        
        # Check if upper-right corner of i is inside j.
        if i_coords[2] >= j_coords[0]:
            if i_coords[2] <= j_coords[2]:
                if i_coords[1] >= j_coords[1]:
                    if i_coords[1] <= j_coords[3]:
                        break
                        
        # Check if lower-right corner of i is inside j.
        if i_coords[2] >= j_coords[0]:
            if i_coords[2] <= j_coords[2]:
                if i_coords[3] >= j_coords[1]:
                    if i_coords[3] <= j_coords[3]:
                        break
                        
        if j == file_list[-1]:
            lst.append(i)
    
    if i % 100 == 0:
        print(f'We are through iteration {i} of 1,233 after {round(time.time()-t0,3)} seconds.')
        
print(f'Our list of candidates is {lst}.')

We are through iteration 0 of 1,233 after 0.004 seconds.
We are through iteration 100 of 1,233 after 0.182 seconds.
We are through iteration 200 of 1,233 after 0.358 seconds.
We are through iteration 300 of 1,233 after 0.512 seconds.
We are through iteration 400 of 1,233 after 0.66 seconds.
We are through iteration 500 of 1,233 after 0.813 seconds.
We are through iteration 600 of 1,233 after 0.953 seconds.
We are through iteration 700 of 1,233 after 1.09 seconds.
We are through iteration 800 of 1,233 after 1.216 seconds.
We are through iteration 900 of 1,233 after 1.329 seconds.
We are through iteration 1000 of 1,233 after 1.409 seconds.
We are through iteration 1100 of 1,233 after 1.474 seconds.
We are through iteration 1200 of 1,233 after 1.501 seconds.
Our list of candidates is [15, 28, 49, 90, 98, 135, 138, 140, 144, 153, 158, 164, 186, 209, 212, 214, 232, 234, 243, 244, 245, 250, 251, 261, 268, 272, 276, 292, 303, 315, 320, 330, 332, 345, 348, 349, 352, 354, 369, 370, 375, 379, 39

In [245]:
lst

[15,
 28,
 49,
 90,
 98,
 135,
 138,
 140,
 144,
 153,
 158,
 164,
 186,
 209,
 212,
 214,
 232,
 234,
 243,
 244,
 245,
 250,
 251,
 261,
 268,
 272,
 276,
 292,
 303,
 315,
 320,
 330,
 332,
 345,
 348,
 349,
 352,
 354,
 369,
 370,
 375,
 379,
 398,
 403,
 404,
 412,
 416,
 421,
 426,
 427,
 428,
 430,
 432,
 435,
 450,
 454,
 459,
 463,
 473,
 476,
 477,
 483,
 484,
 485,
 489,
 498,
 503,
 506,
 509,
 512,
 513,
 514,
 519,
 526,
 530,
 537,
 538,
 545,
 546,
 553,
 559,
 564,
 567,
 569,
 571,
 577,
 582,
 587,
 594,
 602,
 608,
 610,
 611,
 612,
 614,
 615,
 617,
 619,
 628,
 629,
 630,
 632,
 636,
 637,
 638,
 640,
 641,
 642,
 646,
 647,
 652,
 654,
 658,
 659,
 665,
 667,
 668,
 669,
 674,
 677,
 688,
 695,
 696,
 699,
 700,
 702,
 703,
 705,
 708,
 711,
 712,
 718,
 719,
 720,
 721,
 723,
 724,
 726,
 727,
 728,
 729,
 732,
 742,
 746,
 753,
 754,
 758,
 761,
 762,
 763,
 764,
 766,
 771,
 774,
 777,
 779,
 780,
 781,
 783,
 784,
 785,
 786,
 789,
 791,
 792,
 794,
 795,
 79

In [272]:
for k in lst:
    coord_1, coord_2, coord_3, coord_4 = transform(file_list[k])
    count = 0
    for x in range(coord_1, coord_3 + 1):
        for y in range(coord_2, coord_4 + 1):
            if pixel_dict[x,y] == 1:
                count += 1
    if count == (coord_3 - coord_1 + 1) * (coord_4 - coord_2 + 1):
        print(k)

330


In [273]:
transform(file_list[330])

(670, 222, 692, 247)

In [274]:
file_list[330]

'#331 @ 670,222: 23x26\n'

#### How might I improve this?
The only way I can imagine improving this right now is to manage to stop counting values for a pixel once we've noticed overlap. This would mean fewer computations. However, when I rearranged the order of my `for` loops, that became much harder.

I also wonder if I could just add 1 to every value between the upper-left and bottom-right corners directly rather than iterating through each pixel repeatedly.

### Resources Used
- https://stackoverflow.com/questions/1059559/split-strings-with-multiple-delimiters
- https://stackoverflow.com/questions/2793324/is-there-a-simple-way-to-delete-a-list-element-by-value